<a href="https://www.kaggle.com/baranozrtc/wine-quality-74-accuracy?scriptVersionId=83966109" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report,f1_score,roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score,StratifiedKFold
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

In [ ]:
data = pd.read_csv("../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.isna().sum()

# ****EDA****

In [ ]:
data1 = data.groupby("quality").mean()

In [ ]:
data1.head()

In [ ]:
sns.barplot(x= data1.index,y = "alcohol", data = data1)

In [ ]:
sns.barplot(x= data1.index,y = "chlorides", data = data1)

In [ ]:
data.corr()

In [ ]:
sns.heatmap(data.corr(),annot=True)

# **Outliers**

In [ ]:
df = data.copy()

In [ ]:
df.columns

In [ ]:
cols = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']

for i in cols:
    mean= df[i].mean()
    std = df[i].std()
    num = mean + 3*std
    num2= mean - 3*std
    df = df[(df[i] <= num) & (df[i] >= mean - 3*std)]

In [ ]:
df = df.drop(["fixed acidity","free sulfur dioxide","volatile acidity","citric acid"],axis = 1)

In [ ]:
sns.heatmap(df.corr(),annot=True)

In [ ]:
data.info()

In [ ]:
df.info()

# **Train-Test Split**

In [ ]:
y = df.quality

In [ ]:
df = df.drop("quality", axis = 1 )

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=41)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

# **Scailing**

In [ ]:
cols = df.columns

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test =scaler.transform(X_test) 

In [ ]:
X_train = pd.DataFrame(data = X_train, columns=cols)
X_test = pd.DataFrame(data = X_test, columns=cols)

In [ ]:
X_train

# **Modelling**

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
parameters = {"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}

In [ ]:
logmodel = LogisticRegression(max_iter = 1000)

In [ ]:
rsc = RandomizedSearchCV(estimator=logmodel,param_distributions= parameters ,cv=5,) 

rsc.fit(X_train,y_train)
print("tuned hpyerparameters :(best parameters) ",rsc.best_params_)
print("Accuracy :",rsc.best_score_)

In [ ]:
logmodel = LogisticRegression(max_iter = 1000)

In [ ]:
from xgboost import XGBClassifier

In [ ]:
estimator = XGBClassifier(random_state = 42)

In [ ]:
parameters = {
    'max_depth': range (6,36 , 6),
    'n_estimators': range(10, 50, 10),
    'learning_rate': [0.01,0.1,0.3,0.5,0.8,1,2],
     'penalty':['l2', 'l1']
    
}

In [ ]:
rsc = RandomizedSearchCV(estimator=estimator,param_distributions= parameters ,cv=5, n_iter = 6, verbose=1, scoring='accuracy')

In [ ]:
rsc.fit(X_train,y_train)

In [ ]:
rsc.best_params_

In [ ]:
rsc.best_score_

In [ ]:
xgb = XGBClassifier(
   max_depth= 30,
    n_estimators= 30,
    learning_rate= 0.8,
    penalty = "l2",
    random_state = 42
)

In [ ]:
X_train.columns

In [ ]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS


# Feature Selection

In [ ]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS


efs = EFS(xgb, 
           min_features=1,
           max_features=7,
           scoring='accuracy',
           print_progress=True,
           cv=5)

feature_names = ('residual sugar', 'chlorides', 'total sulfur dioxide', 'density', 'pH',
       'sulphates', 'alcohol')
efs = efs.fit(X_train, y_train, custom_feature_names=feature_names)

print('Best accuracy score: %.2f' % efs.best_score_)
print('Best subset (indices):', efs.best_idx_)
print('Best subset (corresponding names):', efs.best_feature_names_)

In [ ]:
pd.DataFrame.from_dict(efs.get_metric_dict()).T.sort_values(by ="avg_score",ascending = False).head(15)

In [ ]:
xgb.fit(X_train,y_train)

In [ ]:
y_pred = xgb.predict(X_test)

In [ ]:
print("accuracy:", accuracy_score(y_test,y_pred))

In [ ]:
feature_important = xgb.get_booster().get_score(importance_type='weight')
keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
data.nlargest(50, columns="score").plot(kind='barh', figsize = (20,10)) ## plot top 40 features

In [ ]:
pred = pd.DataFrame({"ypred":y_pred,"ytest" : y_test})

In [ ]:
pred["difference"] = pred.ytest-pred.ypred


In [ ]:
pred